In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

In [3]:
def build_generator(hidden1, hidden2, hidden3):
    model = Sequential()
    model.add(Dense(hidden1, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)

In [4]:
def trainGAN(gen_hidden1, gen_hidden2, gen_hidden3):
    batch_size = 256
    epochs = 7000
    optimizer = Adam(0.0002, 0.5)
    
    dataframe = pd.read_csv('satan.csv').sample(9000) # sample 100 data points randomly from the csv
    
    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    #to visually judge results
    print("Real satan attacks:")
    print(dataset[:2])
    
    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # build the discriminator portion
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # build the generator portion
    generator = build_generator(gen_hidden1, gen_hidden2, gen_hidden3)
    
    #input and output of our combined model
    z = Input(shape=(41,))
    attack = generator(z)
    validity = discriminator(attack)
    
    # build combined model from generator and discriminator
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #break condition for training (when diverging)
    loss_increase_count = 0;
    prev_g_loss = 0;
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]
        
        # generate a matrix of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 41))
        
        # create an array of generated attacks
        gen_attacks = generator.predict(noise)
        
        # loss functions, based on what metrics we specify at model compile time
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # generator loss function
        g_loss = combined.train_on_batch(noise, valid)
        
        if epoch % 100 == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss change: %.3f, Loss increases: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        prev_g_loss = g_loss
            
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
            
        if epoch % 20 == 0:
            f = open("GANresultsSatan.txt", "a")
            np.savetxt("GANresultsSatan.txt", gen_attacks, fmt="%.0f")
            f.close()
            
    # peek at our results
    results = np.loadtxt("GANresultsSatan.txt")
    print("Generated Satan attacks: ")
    print(results[:2])
        
        


In [5]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset

dataframe = pd.read_csv("normalAndSatan.csv")#, header=True) 

# samples 10000 random data points from 500k
dataframe = dataframe.sample(n=15000)
# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
indices_of_satan = np.where(attack_labels == 'satan.')
satan_index = indices_of_satan[0]
dataset = dataframe_encoded.values

print(attack_labels)
print(satan_index)

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


['normal.' 'satan.']
[1]


In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
#print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors
print(num_of_classes)


2


In [7]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 10
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #optimizer=adam
    return model

In [8]:
#for i in range(0,10):
estimator = KerasClassifier(build_fn=baseline_model, epochs=32, batch_size=200, verbose=2)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
y_pred = cross_val_predict(estimator, X, dummy_y, cv=kfold)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

trained_classifier = estimator.fit(X, Y)
print(type(estimator))

cm = confusion_matrix(Y, y_pred)
print(cm)
print("total: " + str(cm.sum()))
print("accuracy: " + str(np.trace(cm) / cm.sum()))
print("Matthews correlation coefficient: " + str(matthews_corrcoef(Y, y_pred)))



print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

f = open("discriminatorResults.txt", "a+")
f.write("TP: %d, FP: %d, FN: %d, TN: %d\n" % (cm[0][0], cm[0][1], cm[1][0], cm[1][1]))
f.close()


Epoch 1/32
 - 0s - loss: 2.1421 - acc: 0.7864
Epoch 2/32
 - 0s - loss: 0.1283 - acc: 0.9824
Epoch 3/32
 - 0s - loss: 0.0535 - acc: 0.9908
Epoch 4/32
 - 0s - loss: 0.0262 - acc: 0.9935
Epoch 5/32
 - 0s - loss: 0.0213 - acc: 0.9948
Epoch 6/32
 - 0s - loss: 0.0196 - acc: 0.9950
Epoch 7/32
 - 0s - loss: 0.0179 - acc: 0.9957
Epoch 8/32
 - 0s - loss: 0.0181 - acc: 0.9953
Epoch 9/32
 - 0s - loss: 0.0164 - acc: 0.9961
Epoch 10/32
 - 0s - loss: 0.0156 - acc: 0.9965
Epoch 11/32
 - 0s - loss: 0.0150 - acc: 0.9969
Epoch 12/32
 - 0s - loss: 0.0149 - acc: 0.9970
Epoch 13/32
 - 0s - loss: 0.0141 - acc: 0.9974
Epoch 14/32
 - 0s - loss: 0.0141 - acc: 0.9970
Epoch 15/32
 - 0s - loss: 0.0144 - acc: 0.9969
Epoch 16/32
 - 0s - loss: 0.0131 - acc: 0.9974
Epoch 17/32
 - 0s - loss: 0.0127 - acc: 0.9978
Epoch 18/32
 - 0s - loss: 0.0128 - acc: 0.9979
Epoch 19/32
 - 0s - loss: 0.0125 - acc: 0.9979
Epoch 20/32
 - 0s - loss: 0.0122 - acc: 0.9979
Epoch 21/32
 - 0s - loss: 0.0120 - acc: 0.9980
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 0.0081 - acc: 0.9979
Epoch 18/32
 - 0s - loss: 0.0080 - acc: 0.9975
Epoch 19/32
 - 0s - loss: 0.0078 - acc: 0.9977
Epoch 20/32
 - 0s - loss: 0.0075 - acc: 0.9977
Epoch 21/32
 - 0s - loss: 0.0075 - acc: 0.9978
Epoch 22/32
 - 0s - loss: 0.0073 - acc: 0.9978
Epoch 23/32
 - 0s - loss: 0.0075 - acc: 0.9975
Epoch 24/32
 - 0s - loss: 0.0071 - acc: 0.9979
Epoch 25/32
 - 0s - loss: 0.0071 - acc: 0.9979
Epoch 26/32
 - 0s - loss: 0.0069 - acc: 0.9980
Epoch 27/32
 - 0s - loss: 0.0067 - acc: 0.9980
Epoch 28/32
 - 0s - loss: 0.0067 - acc: 0.9980
Epoch 29/32
 - 0s - loss: 0.0064 - acc: 0.9981
Epoch 30/32
 - 0s - loss: 0.0063 - acc: 0.9982
Epoch 31/32
 - 0s - loss: 0.0060 - acc: 0.9986
Epoch 32/32
 - 0s - loss: 0.0058 - acc: 0.9983
Epoch 1/32
 - 0s - loss: 8.0889 - acc: 0.4956
Epoch 2/32
 - 0s - loss: 8.0727 - acc: 0.4984
Epoch 3/32
 - 0s - loss: 8.0700 - acc: 0.4984
Epoch 4/32
 - 0s - loss: 7.0311 - acc: 0.5055
Epoch 5/32
 - 0s - loss: 0.3018 - acc: 0.9612
Epoch 6/32
 - 0s -

Epoch 1/32
 - 0s - loss: 1.7271 - acc: 0.8801
Epoch 2/32
 - 0s - loss: 0.6485 - acc: 0.9543
Epoch 3/32
 - 0s - loss: 0.2186 - acc: 0.9757
Epoch 4/32
 - 0s - loss: 0.0795 - acc: 0.9899
Epoch 5/32
 - 0s - loss: 0.0681 - acc: 0.9915
Epoch 6/32
 - 0s - loss: 0.0616 - acc: 0.9924
Epoch 7/32
 - 0s - loss: 0.0560 - acc: 0.9938
Epoch 8/32
 - 0s - loss: 0.0514 - acc: 0.9941
Epoch 9/32
 - 0s - loss: 0.0440 - acc: 0.9953
Epoch 10/32
 - 0s - loss: 0.0389 - acc: 0.9959
Epoch 11/32
 - 0s - loss: 0.0357 - acc: 0.9962
Epoch 12/32
 - 0s - loss: 0.0339 - acc: 0.9964
Epoch 13/32
 - 0s - loss: 0.0325 - acc: 0.9964
Epoch 14/32
 - 0s - loss: 0.0312 - acc: 0.9965
Epoch 15/32
 - 0s - loss: 0.0304 - acc: 0.9967
Epoch 16/32
 - 0s - loss: 0.0292 - acc: 0.9969
Epoch 17/32
 - 0s - loss: 0.0285 - acc: 0.9967
Epoch 18/32
 - 0s - loss: 0.0274 - acc: 0.9966
Epoch 19/32
 - 0s - loss: 0.0261 - acc: 0.9967
Epoch 20/32
 - 0s - loss: 0.0250 - acc: 0.9970
Epoch 21/32
 - 0s - loss: 0.0244 - acc: 0.9970
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 0.0310 - acc: 0.9967
Epoch 18/32
 - 0s - loss: 0.0310 - acc: 0.9967
Epoch 19/32
 - 0s - loss: 0.0298 - acc: 0.9968
Epoch 20/32
 - 0s - loss: 0.0292 - acc: 0.9972
Epoch 21/32
 - 0s - loss: 0.0289 - acc: 0.9971
Epoch 22/32
 - 0s - loss: 0.0283 - acc: 0.9973
Epoch 23/32
 - 0s - loss: 0.0277 - acc: 0.9972
Epoch 24/32
 - 0s - loss: 0.0276 - acc: 0.9975
Epoch 25/32
 - 0s - loss: 0.0271 - acc: 0.9975
Epoch 26/32
 - 0s - loss: 0.0267 - acc: 0.9976
Epoch 27/32
 - 0s - loss: 0.0264 - acc: 0.9977
Epoch 28/32
 - 0s - loss: 0.0264 - acc: 0.9976
Epoch 29/32
 - 0s - loss: 0.0260 - acc: 0.9975
Epoch 30/32
 - 0s - loss: 0.0260 - acc: 0.9975
Epoch 31/32
 - 0s - loss: 0.0256 - acc: 0.9977
Epoch 32/32
 - 0s - loss: 0.0257 - acc: 0.9976
Epoch 1/32
 - 0s - loss: 6.0534 - acc: 0.5811
Epoch 2/32
 - 0s - loss: 0.3258 - acc: 0.9744
Epoch 3/32
 - 0s - loss: 0.2845 - acc: 0.9874
Epoch 4/32
 - 0s - loss: 0.2576 - acc: 0.9890
Epoch 5/32
 - 0s - loss: 0.2351 - acc: 0.9893
Epoch 6/32
 - 0s -

In [ ]:
"""
f = open("GeneratorHypersAbove50percentAccuracy.txt", "w")
f.write("""""" Hidden layer counts for Generator model that resulted in over 50% generated attacks labeled correctly:
    ------------------------------------------------------------------------------------------------
    """""")
f.close()
"""

while(1):
    # generate random numbers for the hidden layer sizes of our generator
    gen_hidden1 = np.random.randint(1, 101)
    gen_hidden2 = np.random.randint(1, 101)
    gen_hidden3 = np.random.randint(1, 101)
    
    i = 0
    
    
    # train 5 times on each setup, in case we get unlucky initalization on an otherwise good setup
    while i < 5:
        # create a unique filename in case we want to store the results (good accuracy)
        result_filename = "GANresultsSatan%.0f%.0f%.0fiter%.0f.txt" % (gen_hidden1, gen_hidden2, gen_hidden3, i)

        trainGAN(gen_hidden1, gen_hidden2, gen_hidden3)
        
        # load generate attacks from file
        results = np.loadtxt("GANresultsSatan.txt")

        # predict attack lables (as encoded integers)
        y_pred = estimator.predict(results)
        print(y_pred)

        # create appropriate labels for our generated neptune attacks
        satan_labels = np.full((len(results),), satan_index[0])

        # convert integer labels back to string, get all unique strings and their count
        predicted_as_label = attack_labels[y_pred]
        unique_labels = np.unique(predicted_as_label)

        for label in unique_labels:
            print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))
    
        print()
        # create a confusion matrix of the results
        cm = confusion_matrix(satan_labels, y_pred)
        
        accuracy = np.trace(cm) / cm.sum()
        print(cm)
        print("total: " + str(cm.sum()))
        print("accuracy: " + str(accuracy))
        
        if accuracy > .2:
            f = open("GeneratorHypersAbove50percentAccuracySatan.txt", "a")
            f.write("""
            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: %.3f
Generator hidden layer 1 size: %.0f
Generator hidden layer 2 size: %.0f
Generator hidden layer 3 size: %.0f
Iteration %.0f
Result file name: %s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~""" % (accuracy, gen_hidden1, gen_hidden2, gen_hidden3, i, result_filename))
            f.close()
            
            f = open(result_filename, "w")
            f.close()
            np.savetxt(result_filename, results, fmt="%.0f")
        
        i = i + 1
            


Real satan attacks:
[[  0   1  41   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 504   0   0   0   1   1   0   1   0 188   0   0  99   0
    0   7   0  93   7   0]
 [  0   1  46   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0 504   0   0   0   1   1   0   1   0 188   0   0  99   0
    0  14   0  85   7   0]]
0 [D loss: 0.361672, acc.: 59.77%] [G loss: 0.652551] [Loss change: 0.653, Loss increases: 0]
100 [D loss: 0.542282, acc.: 50.59%] [G loss: 0.457419] [Loss change: 0.004, Loss increases: 0]
200 [D loss: 0.565364, acc.: 50.20%] [G loss: 0.423014] [Loss change: -0.003, Loss increases: 2]
300 [D loss: 0.570161, acc.: 50.20%] [G loss: 0.417794] [Loss change: -0.004, Loss increases: 1]
400 [D loss: 0.565888, acc.: 50.39%] [G loss: 0.421919] [Loss change: -0.001, Loss increases: 1]
500 [D loss: 0.566173, acc.: 50.39%] [G loss: 0.434476] [Loss change: -0.001, Loss increases: 2]
600 [D loss: 0.536576, acc.: 51.17%] [G loss: 0.452

0 [D loss: 0.373679, acc.: 51.76%] [G loss: 0.530985] [Loss change: 0.531, Loss increases: 0]
100 [D loss: 0.628827, acc.: 50.39%] [G loss: 0.391195] [Loss change: 0.020, Loss increases: 0]
200 [D loss: 0.669749, acc.: 50.20%] [G loss: 0.344660] [Loss change: -0.002, Loss increases: 0]
300 [D loss: 0.662020, acc.: 50.00%] [G loss: 0.350489] [Loss change: 0.008, Loss increases: 1]
400 [D loss: 0.665818, acc.: 49.80%] [G loss: 0.349786] [Loss change: -0.001, Loss increases: 0]
500 [D loss: 0.655928, acc.: 50.20%] [G loss: 0.352802] [Loss change: -0.003, Loss increases: 1]
600 [D loss: 0.622422, acc.: 50.20%] [G loss: 0.368968] [Loss change: 0.010, Loss increases: 0]
700 [D loss: 0.615471, acc.: 50.39%] [G loss: 0.381512] [Loss change: 0.002, Loss increases: 1]
800 [D loss: 0.609128, acc.: 50.78%] [G loss: 0.391706] [Loss change: 0.007, Loss increases: 0]
900 [D loss: 0.617064, acc.: 50.59%] [G loss: 0.385591] [Loss change: 0.003, Loss increases: 0]
1000 [D loss: 0.645079, acc.: 50.39%] [

0 [D loss: 8.205241, acc.: 49.80%] [G loss: 1.019433] [Loss change: 1.019, Loss increases: 0]
100 [D loss: 0.668372, acc.: 50.59%] [G loss: 0.345863] [Loss change: -0.009, Loss increases: 1]
200 [D loss: 1.228056, acc.: 50.39%] [G loss: 0.150472] [Loss change: 0.019, Loss increases: 0]
300 [D loss: 1.431589, acc.: 49.80%] [G loss: 0.106571] [Loss change: 0.011, Loss increases: 0]
400 [D loss: 1.554864, acc.: 49.61%] [G loss: 0.077791] [Loss change: -0.009, Loss increases: 1]
500 [D loss: 1.516827, acc.: 50.00%] [G loss: 0.075150] [Loss change: -0.011, Loss increases: 5]
600 [D loss: 1.549345, acc.: 50.00%] [G loss: 0.075997] [Loss change: -0.003, Loss increases: 2]
700 [D loss: 1.557241, acc.: 50.00%] [G loss: 0.070832] [Loss change: -0.002, Loss increases: 0]
800 [D loss: 1.521999, acc.: 49.80%] [G loss: 0.075173] [Loss change: -0.001, Loss increases: 1]
900 [D loss: 1.484447, acc.: 49.61%] [G loss: 0.081987] [Loss change: 0.006, Loss increases: 0]
1000 [D loss: 1.471696, acc.: 49.80%

0 [D loss: 8.298325, acc.: 49.41%] [G loss: 0.846500] [Loss change: 0.846, Loss increases: 0]
100 [D loss: 0.788870, acc.: 50.00%] [G loss: 0.275492] [Loss change: 0.007, Loss increases: 0]
200 [D loss: 1.272814, acc.: 50.00%] [G loss: 0.133209] [Loss change: -0.008, Loss increases: 0]
300 [D loss: 1.547228, acc.: 50.00%] [G loss: 0.082034] [Loss change: -0.001, Loss increases: 0]
400 [D loss: 1.785368, acc.: 50.39%] [G loss: 0.053248] [Loss change: 0.000, Loss increases: 2]
500 [D loss: 1.786091, acc.: 50.00%] [G loss: 0.046098] [Loss change: -0.001, Loss increases: 1]
600 [D loss: 1.891962, acc.: 49.61%] [G loss: 0.045038] [Loss change: 0.006, Loss increases: 0]
700 [D loss: 1.754454, acc.: 50.00%] [G loss: 0.046234] [Loss change: 0.003, Loss increases: 0]
800 [D loss: 1.782065, acc.: 49.80%] [G loss: 0.043995] [Loss change: -0.004, Loss increases: 2]
900 [D loss: 1.745751, acc.: 49.80%] [G loss: 0.047550] [Loss change: -0.007, Loss increases: 1]
1000 [D loss: 1.724537, acc.: 50.00%]

700 [D loss: 1.443231, acc.: 49.80%] [G loss: 0.075997] [Loss change: -0.004, Loss increases: 0]
800 [D loss: 1.416602, acc.: 50.00%] [G loss: 0.078975] [Loss change: -0.006, Loss increases: 1]
900 [D loss: 1.401827, acc.: 50.00%] [G loss: 0.085874] [Loss change: 0.007, Loss increases: 0]
1000 [D loss: 1.412245, acc.: 50.00%] [G loss: 0.084048] [Loss change: 0.004, Loss increases: 1]
Stoping on iteration:  1033
Generated Satan attacks: 
[[10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  1.  0.  5.  1.  1.  0. 16.  0.  0. 20.  0.
   0.  5.  0.  3.  0.]
 [ 0.  0.  4.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0. 11.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

...

0 [D loss: 6.278327, acc.: 28.52%] [G loss: 0.641507] [Loss change: 0.642, Loss increases: 0]
100 [D loss: 0.971121, acc.: 50.00%] [G loss: 0.187973] [Loss change: 0.005, Loss increases: 0]
200 [D loss: 1.281923, acc.: 50.00%] [G loss: 0.108126] [Loss change: 0.002, Loss increases: 1]
300 [D loss: 1.601220, acc.: 50.00%] [G loss: 0.064412] [Loss change: 0.000, Loss increases: 0]
400 [D loss: 1.773576, acc.: 50.00%] [G loss: 0.044243] [Loss change: -0.001, Loss increases: 0]
500 [D loss: 1.933273, acc.: 50.00%] [G loss: 0.040366] [Loss change: -0.000, Loss increases: 0]
600 [D loss: 1.936334, acc.: 49.80%] [G loss: 0.033727] [Loss change: -0.009, Loss increases: 3]
700 [D loss: 1.964780, acc.: 49.80%] [G loss: 0.033409] [Loss change: -0.002, Loss increases: 0]
800 [D loss: 1.941364, acc.: 50.00%] [G loss: 0.033498] [Loss change: 0.000, Loss increases: 0]
900 [D loss: 1.902331, acc.: 50.00%] [G loss: 0.033613] [Loss change: -0.000, Loss increases: 0]
1000 [D loss: 1.887760, acc.: 50.00%]

3000 [D loss: 1.311051, acc.: 49.41%] [G loss: 0.129117] [Loss change: -0.004, Loss increases: 1]
3100 [D loss: 1.280671, acc.: 49.41%] [G loss: 0.137868] [Loss change: -0.001, Loss increases: 0]
3200 [D loss: 1.262959, acc.: 50.00%] [G loss: 0.146379] [Loss change: -0.012, Loss increases: 2]
3300 [D loss: 1.276267, acc.: 49.41%] [G loss: 0.155721] [Loss change: -0.001, Loss increases: 0]
3400 [D loss: 1.251746, acc.: 50.00%] [G loss: 0.155884] [Loss change: -0.003, Loss increases: 0]
3500 [D loss: 1.219076, acc.: 50.00%] [G loss: 0.162053] [Loss change: -0.002, Loss increases: 0]
3600 [D loss: 1.208572, acc.: 49.41%] [G loss: 0.175981] [Loss change: 0.012, Loss increases: 0]
3700 [D loss: 1.131031, acc.: 50.00%] [G loss: 0.191776] [Loss change: 0.015, Loss increases: 0]
3800 [D loss: 1.149554, acc.: 49.22%] [G loss: 0.188926] [Loss change: -0.002, Loss increases: 1]
3900 [D loss: 1.160003, acc.: 50.00%] [G loss: 0.192490] [Loss change: -0.000, Loss increases: 0]
4000 [D loss: 1.097284

0 [D loss: 8.297157, acc.: 49.61%] [G loss: 1.169765] [Loss change: 1.170, Loss increases: 0]
100 [D loss: 0.643028, acc.: 51.37%] [G loss: 0.379368] [Loss change: -0.004, Loss increases: 1]
200 [D loss: 1.041654, acc.: 50.20%] [G loss: 0.191051] [Loss change: -0.010, Loss increases: 1]
300 [D loss: 1.278226, acc.: 50.20%] [G loss: 0.116256] [Loss change: -0.001, Loss increases: 0]
400 [D loss: 1.540731, acc.: 50.00%] [G loss: 0.090681] [Loss change: 0.019, Loss increases: 0]
500 [D loss: 1.608760, acc.: 49.80%] [G loss: 0.066815] [Loss change: -0.003, Loss increases: 1]
600 [D loss: 1.589985, acc.: 49.61%] [G loss: 0.070618] [Loss change: 0.010, Loss increases: 0]
700 [D loss: 1.652942, acc.: 50.00%] [G loss: 0.054613] [Loss change: -0.003, Loss increases: 0]
800 [D loss: 1.576186, acc.: 49.80%] [G loss: 0.057288] [Loss change: -0.002, Loss increases: 1]
900 [D loss: 1.561883, acc.: 50.00%] [G loss: 0.063323] [Loss change: 0.006, Loss increases: 0]
1000 [D loss: 1.572755, acc.: 50.00%

200 [D loss: 1.089138, acc.: 50.00%] [G loss: 0.182070] [Loss change: -0.003, Loss increases: 1]
300 [D loss: 1.297898, acc.: 50.00%] [G loss: 0.118795] [Loss change: -0.005, Loss increases: 1]
400 [D loss: 1.556906, acc.: 50.00%] [G loss: 0.092303] [Loss change: -0.000, Loss increases: 0]
500 [D loss: 1.655635, acc.: 49.80%] [G loss: 0.069218] [Loss change: 0.000, Loss increases: 1]
600 [D loss: 1.728207, acc.: 50.00%] [G loss: 0.056940] [Loss change: -0.006, Loss increases: 1]
700 [D loss: 1.667023, acc.: 49.61%] [G loss: 0.067807] [Loss change: 0.013, Loss increases: 0]
Stoping on iteration:  707
Generated Satan attacks: 
[[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0. 10.  0.  0.  1.  0.
   0.  0.  0.  3.  0.]]
[1 1 1 1 1 1 1 1 1 1 1 1

0 [D loss: 2.394285, acc.: 34.96%] [G loss: 0.642427] [Loss change: 0.642, Loss increases: 0]
100 [D loss: 0.841954, acc.: 50.39%] [G loss: 0.263317] [Loss change: -0.001, Loss increases: 0]
200 [D loss: 1.268271, acc.: 50.00%] [G loss: 0.119672] [Loss change: -0.009, Loss increases: 0]
300 [D loss: 1.474227, acc.: 50.00%] [G loss: 0.092247] [Loss change: -0.001, Loss increases: 1]
400 [D loss: 1.609757, acc.: 50.00%] [G loss: 0.072254] [Loss change: 0.009, Loss increases: 0]
500 [D loss: 1.674021, acc.: 50.00%] [G loss: 0.059420] [Loss change: 0.004, Loss increases: 0]
600 [D loss: 1.683630, acc.: 50.00%] [G loss: 0.053452] [Loss change: -0.006, Loss increases: 2]
700 [D loss: 1.692502, acc.: 50.00%] [G loss: 0.045894] [Loss change: -0.000, Loss increases: 0]
800 [D loss: 1.688529, acc.: 50.00%] [G loss: 0.053212] [Loss change: 0.002, Loss increases: 1]
Stoping on iteration:  884
Generated Satan attacks: 
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7. 0.
  0. 0